In [ ]:
######imports

import os

# Add an indented block of code here
# For example:
print("Hello, world!")
songs = util.import_data(file_loc)

def remove_sections(text):
    # Define a regular expression pattern to match the song sections and their variations
    pattern = r"\[(Verse|Intro|Chorus|Interlude|Outro).*?\]"
    
    # Replace the matched patterns with an empty string
    cleaned_text = re.sub(pattern, "", text)
    
    return cleaned_text.strip()



songs['Lyrics'] = songs['Lyrics'].apply(remove_sections)
print(f"Lyrics after remove_sections: {songs['Lyrics'][0]}")

songs['Lyrics'] = songs['Lyrics'].apply(util.handle_special)
print(songs['Lyrics'][0])   
      
songs['Lyrics'] = songs['Lyrics'].apply(util.remove_non_ascii_and_print)
print(songs['Lyrics'][0])


contractions_dict = {
    "im": "i am",
    "youre": "you are",
    "hes": "he is",
    "shes": "she is",
    "its": "it is",
    "were": "we are",
    "theyre": "they are",
    "ive": "i have",
    "youve": "you have",
    "weve": "we have",
    "theyve": "they have",
    "isnt": "is not",
    "arent": "are not",
    "wasnt": "was not",
    "werent": "were not",
    "havent": "have not",
    "hasnt": "has not",
    "hadnt": "had not",
    "wont": "will not",
    "wouldnt": "would not",
    "dont": "do not",
    "doesnt": "does not",
    "didnt": "did not",
    "cant": "cannot",
    "couldnt": "could not",
    "shouldnt": "should not",
    # Add more contractions and their expansions as needed
}
import re

def expand_contractions(text, contractions_dict):
    def expand_match(contraction):
        match = contraction.group(0)
        first_word, rest = re.match(r"(\w+)(.*)", match, re.IGNORECASE).groups()
        expanded_contraction = contractions_dict.get(first_word.lower(), first_word.lower()) + rest
        return expanded_contraction

    contractions_pattern = re.compile(r'\b({})\b'.format('|'.join(contractions_dict.keys())), flags=re.IGNORECASE)
    expanded_text = contractions_pattern.sub(expand_match, text)
    return expanded_text











songs['Lyrics'] = songs['Lyrics'].apply(expand_contractions, args=(contractions_dict,))
print(f"Lyrics after expanding contractions: {songs['Lyrics'][0]}")








tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens = {'pad_token': '<PAD>'}
special_tokens_dict = {'additional_special_tokens': ['<startsong>', '<endsong>']}

tokenizer.add_special_tokens(special_tokens)
tokenizer.add_special_tokens(special_tokens_dict)

class SongDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        for txt in txt_list:
            encodings_dict = tokenizer('<startsong> '+ txt + ' <endsong>', truncation=True, max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]


# Assuming `songs` is a list containing all your song lyrics
dataset = SongDataset(songs['Lyrics'], tokenizer, max_length=512)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True) 


from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

# Move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



from transformers import AdamW, get_linear_schedule_with_warmup
epochs = 4
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader) * epochs)



epoch_losses = []
epoch_perplexities = []
# Training loop with gradient accumulation
epochs = 10
gradient_accumulation_steps = 4  # Increase this if facing memory issues
model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    total_loss = 0
    model.zero_grad()  # Move the zero_grad() outside the batch loop for gradient accumulation
    for batch_idx, (input_ids, masks) in enumerate(dataloader):
        input_ids, masks = input_ids.to(device), masks.to(device)
        outputs = model(input_ids, labels=input_ids, attention_mask=masks)
        loss = outputs.loss / gradient_accumulation_steps  # Scale the loss
        loss.backward()
        total_loss += loss.item()
        perplexity = np.exp(loss.item())
        print(f"Batch {batch_idx}/{len(dataloader)} - Loss: {loss.item()} - Perplexity: {perplexity}")
        if (batch_idx + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            model.zero_grad()

        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx}/{len(dataloader)} - Loss: {loss.item()}")

            # Decode and print the input, target, and prediction
            input_tokens = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            target_tokens = tokenizer.decode(input_ids[0], skip_special_tokens=True)  # same as input for LM
            prediction_ids = torch.argmax(outputs.logits, dim=-1)[0]
            prediction_tokens = tokenizer.decode(prediction_ids, skip_special_tokens=True)

            print(f"  Input Sequence: {input_tokens}")
            print(f"  Target Sequence: {target_tokens}")
            print(f"  Prediction: {prediction_tokens}\n")

    # Calculate average loss and perplexity for the epoch
    avg_loss = total_loss / len(dataloader)
    avg_perplexity = np.exp(avg_loss)
    epoch_losses.append(avg_loss)
    epoch_perplexities.append(avg_perplexity)
    print(f"Average Loss: {avg_loss} - Average Perplexity: {avg_perplexity}\n")



In [ ]:

import matplotlib.pyplot as plt

def plot_metrics(epoch_losses, epoch_perplexities):
    epochs_range = range(1, len(epoch_losses) + 1)
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, epoch_losses, label='Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Epochs')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, epoch_perplexities, label='Training Perplexity')
    plt.xlabel('Epochs')
    plt.ylabel('Perplexity')
    plt.title('Training Perplexity Over Epochs')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_metrics(epoch_losses, epoch_perplexities)


model.eval()
prompt = "<startsong>"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

def generate_text_samples(model, tokenizer, prompt, device, num_samples=3):
    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    generated = generated.to(device)

    input_ids = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=device)
    # Define different generation configurations
    generation_configs = [
        {"config": {"temperature": 0.8, "top_k": 50, "top_p": 0.95}, "description": "Temperature"},
        {"config": {"temperature": 1.0, "top_k": 30, "top_p": 0.95}, "description": "Top-K"},
        {"config": {"temperature": 1.0, "top_k": 0, "top_p": 0.85}, "description": "Top-P"},
        {"config": {"temperature": 1.0, "top_k": 50, "top_p": 0.95, "num_beams": 5, "early_stopping": True}, "description": "Beam Search"},
        {"config": {"temperature": 1.0, "top_k": 50, "top_p": 0.95, "repetition_penalty": 2.0}, "description": "No Repetition Penalty"},

        {"config": {"temperature": 0.7, "top_k": 50, "top_p": 0.95, "repetition_penalty": 2.5}, "description": "Temperature with High Repetition Penalty"},
        {"config": {"temperature": 0.9, "top_k": 20, "top_p": 0.85, "repetition_penalty": 2.0, "no_repeat_ngram_size": 2}, "description": "Top-K with N-Gram Repetition Prevention"},
        {"config": {"temperature": 1.0, "top_k": 0, "top_p": 0.8, "repetition_penalty": 2.0, "no_repeat_ngram_size": 3}, "description": "Top-P with N-Gram Repetition Prevention"},
        {"config": {"num_beams": 5, "early_stopping": True, "no_repeat_ngram_size": 3}, "description": "Beam Search with N-Gram Repetition Prevention"},
        {"config": {"temperature": 1.0, "top_k": 50, "top_p": 0.95, "repetition_penalty": 3.0, "no_repeat_ngram_size": 4}, "description": "No Repetition Penalty with Strong N-Gram Prevention"}
    ]
    for config in generation_configs:
        print(f"Generating with {config['description']}...")
        sample_outputs = model.generate(
             input_ids,
            attention_mask=attention_mask,
            do_sample=True,
            max_length=300,
            num_return_sequences=num_samples,
            **config['config']  # Unpack all other parameters from the config dictionary
        )

        for i, sample_output in enumerate(sample_outputs):
            print(f"{config['description']} Sample {i+1}: {tokenizer.decode(sample_output, skip_special_tokens=True)}\n")


# Example usage
generate_text_samples(model, tokenizer, prompt="<startsong>", device=device, num_samples=3)